In [68]:
# Import necessary libraries
import h5py
# Import necessary libraries
import numpy as np
# Import necessary libraries
from sklearn.preprocessing import StandardScaler
# Import necessary libraries
from sklearn.neural_network import MLPClassifier
# Import necessary libraries
from scipy.ndimage import gaussian_filter

# --- Load the dataset ---
fn = "image1-competition.hdf5"
with h5py.File(fn, "r") as f:
    data = f["data"][:]
    tissue_mask = f["tissue_mask"][:]
    classes = f["classes"][:]

H, W, B = data.shape

In [69]:

# --- Train your own model on labeled data ---
mask = (classes != -1)
X_raw = data[mask]
y_labels = classes[mask]

# Standardize features by removing mean and scaling to unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

In [70]:

# your custom model (change this part as much as you want)
# Initialize MLPClassifier model
mlp = MLPClassifier(
    hidden_layer_sizes=(100,50),
    activation='relu',
    solver='adam',
    alpha=1e-4,
    max_iter=300,
    random_state=0,
    verbose=True
)
# Fit the model to the training data
mlp.fit(X_scaled, y_labels)

Iteration 1, loss = 0.53948593
Iteration 2, loss = 0.29146370
Iteration 3, loss = 0.24572814
Iteration 4, loss = 0.21883881
Iteration 5, loss = 0.20299320
Iteration 6, loss = 0.19360363
Iteration 7, loss = 0.17991555
Iteration 8, loss = 0.17037630
Iteration 9, loss = 0.16283280
Iteration 10, loss = 0.15617834
Iteration 11, loss = 0.15223438
Iteration 12, loss = 0.14471920
Iteration 13, loss = 0.13965783
Iteration 14, loss = 0.13807859
Iteration 15, loss = 0.13446105
Iteration 16, loss = 0.12901364
Iteration 17, loss = 0.12057196
Iteration 18, loss = 0.11879294
Iteration 19, loss = 0.11576756
Iteration 20, loss = 0.11329583
Iteration 21, loss = 0.10944337
Iteration 22, loss = 0.10752038
Iteration 23, loss = 0.10556934
Iteration 24, loss = 0.10266548
Iteration 25, loss = 0.09925120
Iteration 26, loss = 0.09911887
Iteration 27, loss = 0.09383159
Iteration 28, loss = 0.08870575
Iteration 29, loss = 0.08743215
Iteration 30, loss = 0.08558999
Iteration 31, loss = 0.08933117
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, random_state=0,
              verbose=True)

In [71]:
# --- Predict on full image (but use their flow) ---
flat_full = data.reshape(-1, B)
flat_scaled = scaler.transform(flat_full)
flat_proba = mlp.predict_proba(flat_scaled)
proba_cube = flat_proba.reshape(H, W, 6)

In [72]:
# --- Smooth ---
smoothed = np.empty_like(proba_cube)
sigma = 1.0
for c in range(6):
    smoothed[:,:,c] = gaussian_filter(proba_cube[:,:,c], sigma)

# Renormalize
smoothed /= smoothed.sum(axis=2, keepdims=True)

# --- Fill non-tissue regions with uniform distribution ---
uniform = np.full((6,), 1/6, dtype=smoothed.dtype)
for i in range(6):
    smoothed[~tissue_mask, i] = uniform[i]

In [74]:
# --- Extract and save exactly like predict_lr.ipynb ---
r0, r1 = 265, 465
c0, c1 = 360, 660
patch = smoothed[r0:r1, c0:c1, :]  # (200, 300, 6)

patch = patch.astype(np.float32)   # downcast

# Create submission DataFrame
np.save("submission.npy", patch)

# Sanity check
# Create submission DataFrame
loaded = np.load("submission.npy")
print("Saved:", loaded.shape, loaded.dtype)

Saved: (200, 300, 6) float32


In [54]:
# ── Cell 7: Export the competition patch as submission.npy ─────────────────

# 1) extract the 200×300 rectangle
r0, r1 = 265, 465   # rows
c0, c1 = 360, 660   # cols
patch = smoothed[r0:r1, c0:c1, :]      # → (200, 300, 6)
assert patch.shape == (200, 300, 6), f"Patch shape mismatch, got {patch.shape}"

# 2) transpose so the server sees (300, 200, 6)
patch = patch.transpose(1, 0, 2)
assert patch.shape == (300, 200, 6), f"After transpose, got {patch.shape}"

# 3) downcast to float32 (halves file size)
patch = patch.astype(np.float32)

# 4) save exactly this 3D array
# Create submission DataFrame
np.save("submission.npy", patch)

# 5) sanity‐check
# Create submission DataFrame
loaded = np.load("submission.npy")
# Create submission DataFrame
print("Wrote submission.npy →", loaded.shape, loaded.dtype)

Wrote submission.npy → (300, 200, 6) float32
